### Code

In [3]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True) # Removed
import numpy as np # Included
import pandas as pd
import csv
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer as DV

#np.set_printoptions(threshold=sys.maxsize)

dir1="/home/ubuntu/dataset/"
#dir1="/Users/ejl334/Downloads/IoT Dataset by UC/Concatenated/Archive/"

from numpy import genfromtxt
benign_traffic = genfromtxt(dir1+'benign_traffic.csv', delimiter=',')
gafgyt_combo = genfromtxt(dir1+'gafgyt_combo.csv', delimiter=',')
# gafgyt_junk = genfromtxt('/home/ubuntu/dataset/gafgyt_junk.csv', delimiter=',')
# gafgyt_scan = genfromtxt('/home/ubuntu/dataset/gafgyt_scan.csv', delimiter=',')
# gafgyt_tcp = genfromtxt('/home/ubuntu/dataset/gafgyt_tcp.csv', delimiter=',')
# gafgyt_udp = genfromtxt('/home/ubuntu/dataset/gafgyt_udp.csv', delimiter=',')
# mirai_ack = genfromtxt('/home/ubuntu/dataset/mirai_ack.csv', delimiter=',')
# mirai_scan = genfromtxt('/home/ubuntu/dataset/mirai_scan.csv', delimiter=',')
# mirai_syn = genfromtxt('/home/ubuntu/dataset/mirai_syn.csv', delimiter=',')
# mirai_udp = genfromtxt('/home/ubuntu/dataset/mirai_udp.csv', delimiter=',')
# mirai_udpplain = genfromtxt('/home/ubuntu/dataset/mirai_udpplain.csv', delimiter=',')

# with open('/home/ubuntu/dataset/benign_traffic.csv', 'r') as csvfile:

#     csvreader = csv.reader(csvfile)
#     next(csvreader,None)
#     output = open('/home/ubuntu/dataset/corrected_benign_traffic.csv', 'wb')
#     writer = csv.writer(output)
#     for row in csvreader:
#         if row[1].isdigit()==True:
#             print(row)
#             #writer.writerow(row)
#     csvfile.close()
#     output.close()

#benign_traffic = pd.read_csv("/home/ubuntu/dataset/benign_traffic.csv")

#benign_traffic.head()
# fx_array=open('Parsed Data/complete_train_rand_xin.npy','rb')
# x_in = np.load(fx_array)

# fy_array=open('Parsed Data/complete_train_rand_yin.npy','rb')
# y_in = np.load(fy_array)

# fxt_array=open('Parsed Data/complete_test_rand_xin.npy','rb')
# xt_in = np.load(fxt_array)

# fyt_array=open('Parsed Data/complete_test_rand_yin.npy','rb')
# yt_in = np.load(fyt_array)


# print("Normal Sample: \n")
# print(" Method: 0\n \
# Content_length:0 \n \
# URL: http://localhost:8080/tienda1/miembros/editar.jsp \n \
# Payload: direccion=Calle+Barrio+Cura-vigo+Vello%2C+134+10%3FD \n")

# print(xt_in[84])
# print("\n")
# print(yt_in[84])
# print("\n")

# print("Abormal Sample:")
# print("\n")
# print(" Method: 0\n \
# Content_length:0 \n \
# URL: http://localhost:8080/tienda1/publico/caracteristicas.jsp \n\n \
# Payload: id=1sessionid%3D12312312%26+username%3D%253C%2573%2563%2572%2569%2570%2574%253E%2564%256F%2563%2575%256D%2565%256E%2574%252E%256C%256F%2563%2561%2574%2569%256F%256E%253D%2527%2568%2574%2574%2570%253A%252F%252F%2561%2574%2574%2561%2563%256B%2565%2572%2568%256F%2573%2574%252E%2565%2578%2561%256D%2570%256C%2565%252F%2563%2567%2569%252D%2562%2569%256E%252F%2563%256F%256F%256B%2569%2565%2573%2574%2565%2561%256C%252E%2563%2567%2569%253F%2527%252B%2564%256F%2563%2575%256D%2565%256E%2574%252E%2563%256F%256F%256B%2569%2565%253C%252F%2573+%2563%2572%2569%2570%2574%253E%3F \n\n \
# Payload Decoded: id 1sessionid 12312312 username script document location http attackerhost example cgi bin cookiesteal cgi document cookie s cript 0 0")
# print("\n")
# print(xt_in[337])
# print("\n")
# print(yt_in[337])

# test_xin_np = np.array([xt_in[84],xt_in[337]])
# test_yin_np = np.array([yt_in[84],yt_in[337]])

# print("Input Test Data:")
# print(type(xt_in))
# print(len(xt_in))
# print(type(xt_in[0]))
# print(len(xt_in[0]))

# print("Input Test Data:")
# print(type(test_xin_np))
# print(len(test_xin_np))
# print(type(test_xin_np[0]))
# print(len(test_xin_np[0]))
# print(test_xin_np[0])

In [4]:
# DELETE COLUMNS WITH NAN
print("========================================================================")
print("***BENIGN TRAFFIC***")
print("========================================================================")
print("***DIMENSIONS***")
numrows = len(benign_traffic)
numcols = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(gafgyt_combo).any(axis=1))
print(nan_rows)

print("***** NAN Removal *****")
benign_traffic = benign_traffic[~np.isnan(benign_traffic).any(axis=1)]

print("***DIMENSIONS***")
numrows_good = len(benign_traffic)
numcols = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_good,numcols))

print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(benign_traffic).any(axis=1))
print(nan_rows)

print("========================================================================")
print("***MALICIOUS TRAFFIC***")
print("========================================================================")
print("***DIMENSIONS***")
malicious_dataset = gafgyt_combo

numrows = len(malicious_dataset)
numcols = len(malicious_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(malicious_dataset).any(axis=1))
print(nan_rows)

print("***** NAN Removal *****")
malicious_dataset = malicious_dataset[~np.isnan(malicious_dataset).any(axis=1)]

print("***DIMENSIONS***")
numrows_bad = len(malicious_dataset)
numcols = len(malicious_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_bad,numcols))
print("***NAN ROWS***")
nan_rows=np.argwhere(np.isnan(malicious_dataset).any(axis=1))
print(nan_rows)

***BENIGN TRAFFIC***
***DIMENSIONS***
Dimensions Rows: 555941 Columns: 115
***NAN ROWS***
[[     0]
 [ 59719]
 [112732]
 [165747]
 [223900]
 [285281]
 [342812]
 [401482]
 [455766]]
***** NAN Removal *****
***DIMENSIONS***
Dimensions Rows: 555932 Columns: 115
***NAN ROWS***
[]
***MALICIOUS TRAFFIC***
***DIMENSIONS***
Dimensions Rows: 515165 Columns: 115
***NAN ROWS***
[[     0]
 [ 59719]
 [112732]
 [165747]
 [223900]
 [285281]
 [342812]
 [401482]
 [455766]]
***** NAN Removal *****
***DIMENSIONS***
Dimensions Rows: 515156 Columns: 115
***NAN ROWS***
[]


In [5]:
benign_traffic[0:2]

array([[1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
        6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
        0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
        1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
        6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
        0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
        1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 1.00000000e+00,
        6.00000000e+01, 0.00000000e+00, 1.00000000e+00, 6.00000000e+01,
        0.00000000e+00, 6.00000000e+01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 6.00000000e+01, 0.00000000e+00,
        6.00000000e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 6.00000000e+01, 0.00000000e+00, 6.00000000e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        6.00000000e+01, 0.00000000e+00, 6.00000000e+01, 0.000000

In [6]:
# DELETE UNWANTED COLUMNS
benign_traffic = np.delete(benign_traffic, [2,5,8,11,14,17,20,23,26,29,\
                                            34,35,41,42,48,49,55,56,62,63,\
                                            66,67,69,70,72,73,75,76,78,79,\
                                            84,85,91,92,98,99,105,106,112,113,], axis=1)
malicious_dataset = np.delete(malicious_dataset, [2,5,8,11,14,17,20,23,26,29,\
                                            34,35,41,42,48,49,55,56,62,63,\
                                            66,67,69,70,72,73,75,76,78,79,\
                                            84,85,91,92,98,99,105,106,112,113,], axis=1)


In [7]:
print("***DIMENSIONS BENIGN***")
numrows_good = len(benign_traffic)
numcols_good = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_good,numcols_good))
print("***DIMENSIONS MALICIOUS***")
numrows_bad = len(malicious_dataset)
numcols_bad = len(malicious_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows_bad,numcols_bad))

***DIMENSIONS BENIGN***
Dimensions Rows: 555932 Columns: 75
***DIMENSIONS MALICIOUS***
Dimensions Rows: 515156 Columns: 75


In [38]:
complete_dataset = np.concatenate((benign_traffic, malicious_dataset), 0)
print("*** COMPLETE DATASET ***")
numrows = len(complete_dataset)
numcols = len(complete_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

*** COMPLETE DATASET ***
Dimensions Rows: 1071088 Columns: 75


In [60]:
# NORMALIZE DATASET
scaler = MinMaxScaler(feature_range=(0, 1))
dataX_ = scaler.fit_transform(complete_dataset)
#dataX_ = scaler.fit_transform(np.reshape(complete_dataset, (numrows, numcols)))
#                                                        (5000,100*75)
X_ = dataX_
#X_ = np.reshape(dataX_, (numrows, numcols))
#X_ = np.reshape(dataX_, (5000, 100, 75))

In [61]:
benign_traffic = X_[:numrows_good, :]
malicious_dataset = X_[numrows_good:, :]

print("*** COMPLETE DATASET ***")
numrows = len(benign_traffic)
numcols = len(benign_traffic[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

print("*** COMPLETE DATASET ***")
numrows = len(malicious_dataset)
numcols = len(malicious_dataset[0])
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

# Clear Variables
del X_
del dataX_


*** COMPLETE DATASET ***
Dimensions Rows: 555932 Columns: 75
*** COMPLETE DATASET ***
Dimensions Rows: 515156 Columns: 75


In [12]:
def rounder(x):
    if (x-int(x) >= 0.5):
        return np.ceil(x)
    else:
        return np.floor(x)

split1 = int(rounder(len(benign_traffic)*0.8))
split2 = int(rounder(len(malicious_dataset)*0.8))
current_bot_data = gafgyt_combo

print(split1)
print(split2)

444746
412125


In [13]:
# benign_traffic = np.delete(benign_traffic, [0,49549,62663,101764,277005,339160,437675,489826,526412], axis=0)
# gafgyt_combo = np.delete(gafgyt_combo, [0,59719,112732,165747,223900,285281,334812,401482,455766], axis=0)
# gafgyt_junk = np.delete(gafgyt_junk, [0,29069,59382,89180,117530,148429,177498,205804,234384], axis=0)
# gafgyt_scan = np.delete(gafgyt_scan, [0,29850,57345,85466,113326,142624,171022,198721,226547], axis=0)
# gafgyt_tcp = np.delete(gafgyt_tcp, [0,92142,187164,288701,381283,485794,575182,672966,761783], axis=0)
# gafgyt_udp = np.delete(gafgyt_udp, [0,105875,210667,314601,420384,524396,629055,739673,843394], axis=0)
# mirai_ack = np.delete(mirai_ack, [0,102196,215482,306606,367161,425159,536640], axis=0)
# mirai_scan = np.delete(mirai_scan, [0,107686,150879,254501,351283,448380,494311], axis=0)
# mirai_syn = np.delete(mirai_syn, [0,122574,239382,357511,423258,485110,610826], axis=0)
# mirai_udp = np.delete(mirai_udp, [0,], axis=0)
# mirai_udpplain = np.delete(mirai_udpplain, [0,], axis=0)

def dimensions_data(x):
    numrows = len(x)
    numcols = len(x[0])
    print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

y_benings = np.zeros(len(benign_traffic),dtype=int)
y_malicious = np.ones(len(malicious_dataset),dtype=int)

benign_pre_train_x = benign_traffic[:split1, :]
print("*** DIMENSIONS benign_pre_train_x***")
dimensions_data(benign_pre_train_x)

benign_pre_test_x = benign_traffic[split1:, :]
print("*** DIMENSIONS benign_pre_test_x***")
dimensions_data(benign_pre_test_x)

benign_pre_train_y = benign_traffic[:split1, :]
print("*** DIMENSIONS benign_pre_train_y***")
dimensions_data(benign_pre_train_y)

benign_pre_test_y = benign_traffic[split1:, :]
print("*** DIMENSIONS benign_pre_test_y***")
dimensions_data(benign_pre_test_y)



malicious_pre_train_x = malicious_dataset[:split2, :]
print("*** DIMENSIONS malicious_pre_train_x***")
dimensions_data(malicious_pre_train_x
               )
malicious_pre_test_x = malicious_dataset[split2:, :]
print("*** DIMENSIONS malicious_pre_test_x***")
dimensions_data(malicious_pre_test_x)

malicious_pre_train_y = malicious_dataset[:split2, :]
print("*** DIMENSIONS malicious_pre_train_y***")
dimensions_data(malicious_pre_train_y)

malicious_pre_test_y = malicious_dataset[split2:, :]
print("*** DIMENSIONS malicious_pre_test_y***")
dimensions_data(malicious_pre_test_y)

train_x = np.concatenate((benign_pre_train_x, malicious_pre_train_x), 0)
print("*** DIMENSIONS - TRAIN_X ***")
dimensions_data(train_x)
    
test_x = np.concatenate((benign_pre_test_x, malicious_pre_test_x), 0)
print("*** DIMENSIONS TEXT_X ***")
dimensions_data(test_x)

train_y = np.concatenate((benign_pre_train_y,malicious_pre_train_y),0)
print("*** DIMENSIONS TRAIN_Y ***")
dimensions_data(train_y)

test_y = np.concatenate((benign_pre_test_y,malicious_pre_test_y),0)
print("*** DIMENSIONS TEST_Y***")
dimensions_data(test_y)

*** DIMENSIONS benign_pre_train_x***
Dimensions Rows: 444746 Columns: 75
*** DIMENSIONS benign_pre_test_x***
Dimensions Rows: 111186 Columns: 75
*** DIMENSIONS benign_pre_train_y***
Dimensions Rows: 444746 Columns: 75
*** DIMENSIONS benign_pre_test_y***
Dimensions Rows: 111186 Columns: 75
*** DIMENSIONS malicious_pre_train_x***
Dimensions Rows: 412125 Columns: 75
*** DIMENSIONS malicious_pre_test_x***
Dimensions Rows: 103031 Columns: 75
*** DIMENSIONS malicious_pre_train_y***
Dimensions Rows: 412125 Columns: 75
*** DIMENSIONS malicious_pre_test_y***
Dimensions Rows: 103031 Columns: 75
*** DIMENSIONS - TRAIN_X ***
Dimensions Rows: 856871 Columns: 75
*** DIMENSIONS TEXT_X ***
Dimensions Rows: 214217 Columns: 75
*** DIMENSIONS TRAIN_Y ***
Dimensions Rows: 856871 Columns: 75
*** DIMENSIONS TEST_Y***
Dimensions Rows: 214217 Columns: 75


In [34]:
#benign_traffic[~np.isnan(benign_traffic).any(axis=1)]
print(benign_traffic[0:2])
numrows = len(benign_traffic)    # 3 rows in your example
numcols = len(benign_traffic[0]) # 2 columns in your example
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.27885519e-01 0.00000000e+00 6.93889390e-18 0.00000000e+00
  0.00000000e+00 4.23431207e-01 0.00000000e+00 6.93889390e-18
  0.00000000e+00 0.00000000e+00 4.05812719e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.73886056e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.84832906e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 6.93889390e-18
  0.00000000e+00 0.00000000e+00 3.70234072e-01 0.00000000e+00
  6.93889390e-18 0.00000000e+00 0.00000000e+00 3.70841959e-01
  0.00000000e+00 6.93889390e-18 0.00000000e+00 0.00000000e+00
  3.6696

In [35]:
print(np.shape(train_x), np.shape(test_x))
print(len(train_y), len(test_y))
print(np.count_nonzero(train_y), np.count_nonzero(test_y))

(856871, 75) (214217, 75)
856871 214217
57138008 14211342


In [92]:
# Vectorize data

#vector_train_x = train_x.reshape((4061, 211, 75))
vector_train_x = train_x.reshape((856871, 1, 75))
print(vector_train_x.shape)
#vector_test_x = test_x.reshape((12601, 17, 75))
vector_test_x = test_x.reshape((214217, 1, 75))
print(vector_test_x.shape)

#vector_train_x = train_x.reshape(856871,75,np.newaxis)
#vector_test_x = test_x.reshape(214217,75,np.newaxis)
print(vector_train_x[0])

(856871, 1, 75)
(214217, 1, 75)
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.27885519e-01 0.00000000e+00 6.93889390e-18 0.00000000e+00
  0.00000000e+00 4.23431207e-01 0.00000000e+00 6.93889390e-18
  0.00000000e+00 0.00000000e+00 4.05812719e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.73886056e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.84832906e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 6.93889390e-18
  0.00000000e+00 0.00000000e+00 3.70234072e-01 0.00000000e+00
  6.93889390e-18 0.00000000e+00 0.00000000e+00 3.70841959e-01
  0.00000000e+00 6.93889390e-18 0.0000

# LSTM MODEL

In [94]:
# create and fit the LSTM network
batch_size = 28
model = Sequential()
model.add(LSTM(28, input_shape=(1,75), return_sequences=True))
model.add(LSTM(28, input_shape=(1,75)))
model.add(Dense(28))
model.add(Dense(75))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model.fit(vector_train_x, train_y, epochs=10, batch_size=batch_size, verbose=2, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
 - 171s - loss: 2.6463e-04 - acc: 0.5736
Epoch 2/10
 - 170s - loss: 3.2337e-05 - acc: 0.6033
Epoch 3/10
 - 168s - loss: 2.5364e-05 - acc: 0.6205
Epoch 4/10
 - 169s - loss: 2.2528e-05 - acc: 0.6419
Epoch 5/10
 - 169s - loss: 1.9008e-05 - acc: 0.6537
Epoch 6/10
 - 169s - loss: 1.6292e-05 - acc: 0.6471
Epoch 7/10
 - 168s - loss: 1.5011e-05 - acc: 0.6526
Epoch 8/10
 - 167s - loss: 1.4688e-05 - acc: 0.6560
Epoch 9/10
 - 154s - loss: 1.4579e-05 - acc: 0.6581
Epoch 10/10
 - 149s - loss: 1.4365e-05 - acc: 0.6624


In [30]:
hm_epochs = 30
n_classes = 2
batch_size = 10
chunk_size = 1 # Changed
n_chunks = 75 # Number of attributes in dataset
rnn_size = 1

x = tf.placeholder("float", shape=[None, n_chunks, chunk_size])
y = tf.placeholder("float")
x_t = tf.placeholder("float", shape=[None, n_chunks, chunk_size])
y_t = tf.placeholder("float")

In [31]:
# x_request = tf.reshape(x, [-1,28,1,1])
# print ("x_request=")
# print (x_request)
#tf.reset_default_graph()

In [33]:
class DataSet(object):
    def __init__(self, requests, labels):
        assert requests.shape[0] == labels.shape[0], ("requests.shape: %s labels.shape: %s" % (requests.shape,
                                                 labels.shape))
        self._num_examples = requests.shape[0]
        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, rows*columns] (assuming depth == 1)
        requests = requests.reshape(requests.shape[0],
                              requests.shape[1])
        self._requests = requests
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            #print("=== Current Perm ===")
            #print("=== index_in_epoch === : %s" % self._index_in_epoch)
            perm = np.arange(self._num_examples)
            #print(perm)
            np.random.shuffle(perm)
            self._requests = self._requests[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._requests[start:end], self._labels[start:end]

current_data = DataSet(train_x, train_y)
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
#     def make_cell():
        
#         lstm_cell = rnn_cell.BasicLSTMCell(rnn_size)
#         return lstm_cell
    
#     lstm_cell = tf.contrib.rnn.MultiRNNCell(
#     [make_cell for _ in range(2)], state_is_tuple=True)
#     print(lstm_cell)
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size) 
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    #Helper
    #Decoder
    # +  = dynamic
    output, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(output[-1],layer['weights']) + layer['biases']

    return output

def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(plogits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(current_data._num_examples/batch_size)):
                epoch_x, epoch_y = current_data.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x.reshape((-1, n_chunks, chunk_size)), y:test_y}))
        #print('Test Abnormal:',accuracy.eval({x_t:test_xin_np.reshape((-1, n_chunks, chunk_size)), y_t:test_yin_np}))

train_neural_network(x)

TypeError: static_rnn() got an unexpected keyword argument 'reuse'